In [1]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import math
import lightgbm as lgb
from collections import Counter  
import time
from scipy.stats import kurtosis,iqr
import seaborn as sns
from scipy import ptp
import time
from tqdm import tqdm
from datetime import timedelta
from sklearn.externals import joblib
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
import os
from sklearn.utils import shuffle
from sklearn.externals import joblib
from mpl_toolkits.mplot3d import Axes3D
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

# 将short_stay和area_passenger_index拆分成多个小文件

In [8]:
# ### 北京市网格人流量指数,200*200网格
# cols = ['date', 'hour', 'Grid_x_1', 'Grid_y_1', 'index_1']
# short_stay = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/shortstay_20200201_20200215.csv', names=cols, encoding='gb2312', sep='\t')
# tem = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/shortstay_20200117_20200131.csv', names=cols, encoding='gb2312', sep='\t')
# short_stay = pd.concat([short_stay, tem], axis=0)

In [9]:
# short_stay['date'] = short_stay['date'].apply(lambda x: ''.join(str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8]))
# short_stay['date'] = pd.to_datetime(short_stay['date'])
# short_stay.iloc[0:5]

In [10]:
# ### 按月，日将short_stay拆分成多个小文件
# ### 定义起始日期为2020-01-17
# start_date = short_stay['date'].iloc[0]-timedelta(days=15)
# for i in tqdm(range(1, 31)):
#     ### 取出一天的数据
#     tem = short_stay[short_stay['date']==start_date]
#     ### 索引重置
#     tem = tem.reset_index(drop=True)
#     ### 保存
#     joblib.dump(tem, '/home/poac/AnomalyDetectionDataset/Block_predict/processed/short_stay_split/'+str(start_date)+'.jl.z')
# #     tem.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/short_stay_split/short_stay_'+str(start_date)+'.csv', index=None)
#     ### 日期加一
#     start_date = start_date + timedelta(days=1)

In [11]:
# del short_stay

In [12]:
# ### 读取区域人流量指数，含标签
# ### 并按照日期将其拆分为多个小文件，分别保存
# area_passenger_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/area_passenger_index.csv')
# area_passenger_index['date'] = pd.to_datetime(area_passenger_index['date'])

In [13]:
# ### 保存
# start_date = area_passenger_index['date'].iloc[0]
# for i in tqdm(range(1, 31)):
#     ### 取出一天的数据
#     tem = area_passenger_index[area_passenger_index['date']==start_date]
#     ### 索引重置
#     tem = tem.reset_index(drop=True)
#     ### 保存
#     tem.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/area_passenger_split/'+str(start_date)+'.csv')
#     ### 日期加一
#     start_date = start_date + timedelta(days=1)

In [14]:
# del area_passenger_index

## 将input_for_sub拆成多个文件

In [21]:
# input_for_sub = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/input_for_sub.csv')
# input_for_sub['date'] = pd.to_datetime(input_for_sub['date'])

In [39]:
# ### 保存
# start_date = input_for_sub['date'].iloc[0]
# for i in tqdm(range(1, 10)):
#     ### 取出一天的数据
#     tem = input_for_sub[input_for_sub['date']==start_date]
#     ### 索引重置
#     tem = tem.reset_index(drop=True)
#     ### 保存
#     tem.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/input_for_sub_split/'+str(start_date)+'.csv')
#     ### 日期加一
#     start_date = start_date + timedelta(days=1)

# 合并area_passenger_index和short_stay

In [1]:
### 读取area_passenger_index所有文件
files = os.listdir('/home/poac/AnomalyDetectionDataset/Block_predict/processed/area_passenger_split/')
area_files = []
for file in files:
    if file[-4:] == '.csv':
        area_files.append(file)

In [2]:
### 读取short_stay所有文件
files = os.listdir('/home/poac/AnomalyDetectionDataset/Block_predict/processed/short_stay_split/')
short_files = []
for file in files:
    if file[-5:] == '.jl.z':
        short_files.append(file)

In [3]:
### 找对area_passenger_index中重点区域中心最近的坐标，并将其index添加
### 先将area_passenger_index和short_stay merge
area_passenger_index_processed = pd.DataFrame()
for file in tqdm(area_files):
    
     ### 读取short_stay
    short_stay = joblib.load('/home/poac/AnomalyDetectionDataset/Block_predict/processed/short_stay_split/'+file[0:-3]+'jl.z')
    short_stay['date'] = short_stay['date'].astype(str)
    ### 处理日期格式
    short_stay['month'] = short_stay['date'].apply(lambda x: int(str(x)[5:7]))
    short_stay['day'] = short_stay['date'].apply(lambda x: int(str(x)[8:]))
    short_stay  = short_stay.drop(['date'], axis=1)
    
    ### 读取area_passenger_index
    area_passenger_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/area_passenger_split/'+file, chunksize=100)
    for chunk in area_passenger_index:
        chunk['date'] = chunk['date'].astype(str)
        ### 处理日期格式
        chunk['month'] = chunk['date'].apply(lambda x: int(str(x)[5:7]))
        chunk['day'] = chunk['date'].apply(lambda x: int(str(x)[8:]))
        chunk  = chunk.drop(['date', 'area_name', 'area_type'], axis=1)
   
        ### merge
        chunk = chunk.merge(short_stay, how='left', on=['month', 'day', 'hour'])
        chunk['distance'] = (chunk['Grid_x']-chunk['Grid_x_1'])**2 + (chunk['Grid_y']-chunk['Grid_y_1'])**2
        ### 按照month,day,hour,distance排序
        chunk= chunk.sort_values(by=['month', 'day', 'hour', 'distance'], ascending=[True, True, True, True])
        ### 按照month,day,hour多列去重复，取第一次出现的数据，实现最小距离的获取
        chunk = chunk.drop_duplicates(subset=['month','day', 'hour'],keep='first')
        ### 处理一个chunk之后合并
        area_passenger_index_processed = pd.concat([area_passenger_index_processed, chunk], axis=0)

In [4]:
### 处理完之后再去一次重
area_passenger_index_processed = area_passenger_index_processed.drop_duplicates(subset=['month','day', 'hour'],keep='first')

In [5]:
### 将处理完的数据保存
area_passenger_index_processed.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/area_passenger_index_processed.csv', index=None)

## 将short_stay和input_for_sub合并

In [1]:
# ### 读取area_passenger_index所有文件
# files = os.listdir('/home/poac/AnomalyDetectionDataset/Block_predict/processed/input_for_sub_split/')
# input_files = []
# for file in files:
#     if file[-4:] == '.csv':
#         input_files.append(file)

In [2]:
# ### input_for_sub，并将其index添加
# ### 先将input_for_sub和short_stay merge
# input_for_sub = pd.DataFrame()
# for file in input_files:
    
#      ### 读取short_stay
#     short_stay = joblib.load('/home/poac/AnomalyDetectionDataset/Block_predict/processed/short_stay_split/'+file[0:-3]+'jl.z')
#     short_stay['date'] = short_stay['date'].astype(str)
#     ### 处理日期格式
#     short_stay['month'] = short_stay['date'].apply(lambda x: int(str(x)[5:7]))
#     short_stay['day'] = short_stay['date'].apply(lambda x: int(str(x)[8:]))
#     short_stay  = short_stay.drop(['date'], axis=1)
    
#     ### 读取input_for_sub
#     input_ = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/input_for_sub_split/'+file, chunksize=100)
#     for chunk in input_:
#         chunk['date'] = chunk['date'].astype(str)
#         ### 处理日期格式
#         chunk['month'] = chunk['date'].apply(lambda x: int(str(x)[5:7]))
#         chunk['day'] = chunk['date'].apply(lambda x: int(str(x)[8:]))
#         chunk  = chunk.drop(['date', 'area_name', 'area_type'], axis=1)
   
#         ### merge
#         chunk = chunk.merge(short_stay, how='left', on=['month', 'day', 'hour'])
#         chunk['distance'] = (chunk['Grid_x']-chunk['Grid_x_1'])**2 + (chunk['Grid_y']-chunk['Grid_y_1'])**2
#         ### 按照month,day,hour,distance排序
#         chunk= chunk.sort_values(by=['month', 'day', 'hour', 'distance'], ascending=[True, True, True, True])
#         ### 按照month,day,hour多列去重复，取第一次出现的数据，实现最小距离的获取
#         chunk = chunk.drop_duplicates(subset=['month','day', 'hour'],keep='first')
#         ### 处理一个chunk之后合并
#         input_for_sub = pd.concat([input_for_sub, chunk], axis=0)

In [3]:
# input_for_sub = input_for_sub.drop_duplicates(subset=['month','day', 'hour'],keep='first')

In [4]:
# ## 保存
# input_for_sub.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/input_for_sub_processed.csv', index=None)

************